In [1]:
from pref_voting.profiles_with_ties import *
from pref_voting.voting_methods import *
from pref_voting.analysis import *
from pref_voting.generate_profiles import *
from pref_voting.generate_weighted_majority_graphs import *

from pref_voting.utility_profiles import UtilityProfile, write_utility_profiles_to_json, read_utility_profiles_from_json
from pref_voting.rankings import Ranking
from pref_voting.generate_utility_profiles import *
from pref_voting.generate_utility_profiles import *
from pref_voting.utility_methods import *

from pref_voting.spatial_profiles import SpatialProfile
from pref_voting.generate_spatial_profiles import *
from pref_voting.utility_functions import *
from pref_voting.probabilistic_methods import *
from tqdm.notebook import tqdm
import nashpy as nash
import numpy as np
import random2 as random
from pref_voting.mappings import _Mapping
from multiprocess import Pool, cpu_count, current_process
from numba import njit, float32
import pickle
import json
from pref_voting.monotonicity_axioms import *
from pref_voting.helper import *
from pref_voting.variable_candidate_axioms import *
from pref_voting.profiles import Profile

In [2]:
import pref_voting
print(pref_voting.__version__)

0.5.33


In [3]:
from prefsampling.ordinal.impartial import impartial


prof = Profile(impartial(10, 2, seed=None))

prof.display()

prof.anonymize().display()

prof.display()

+---+---+---+---+---+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+---+---+---+---+
| 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 |
+---+---+---+---+---+---+---+---+---+---+
+---+---+
| 5 | 5 |
+---+---+
| 0 | 1 |
| 1 | 0 |
+---+---+
+---+---+---+---+---+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+---+---+---+---+
| 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 |
+---+---+---+---+---+---+---+---+---+---+


In [4]:
rng = np.random.default_rng(10012)
print(gamma.rvs(0.8, random_state=rng))
print(gamma.rvs(0.8, random_state=rng))
print(gamma.rvs(0.8, random_state=rng))

0.05041468918240317
1.725090099498795
0.16134003712341477


In [5]:
from prefsampling.ordinal import impartial, impartial_anonymous, urn, plackett_luce, didi, stratification, single_peaked_conitzer, single_peaked_walsh, single_peaked_circle, single_crossing, euclidean, mallows

from prefsampling.core.euclidean import EuclideanSpace


# Given the number m of candidates and a phi\in [0,1] function computes the expected number of swaps in a vote sampled from Mallows model
def find_expected_number_of_swaps(num_candidates, phi):
    res = phi * num_candidates / (1 - phi)
    for j in range(1, num_candidates + 1):
        res = res + (j * (phi**j)) / ((phi**j) - 1)
    return res


# Given the number m of candidates and a absolute number of expected swaps exp_abs, this function returns a value of phi such that in a vote sampled from Mallows model with this parameter the expected number of swaps is exp_abs
def phi_from_relphi(num_candidates, relphi=None, seed=None):

    rng = np.random.default_rng(seed)
    if relphi is None:
        relphi = rng.uniform(0.001, 0.999)
    if relphi == 1:
        return 1
    exp_abs = relphi * (num_candidates * (num_candidates - 1)) / 4
    low = 0
    high = 1
    while low <= high:
        mid = (high + low) / 2
        cur = find_expected_number_of_swaps(num_candidates, mid)
        if abs(cur - exp_abs) < 1e-5:
            return mid
        # If x is greater, ignore left half
        if cur < exp_abs:
            low = mid

        # If x is smaller, ignore right half
        elif cur > exp_abs:
            high = mid

    # If we reach here, then the element was not present
    return -1

# Return a list of phis from the relphi value
def phis_from_relphi(num_candidates, num, relphi=None, seed=None):

    rng = np.random.default_rng(seed)
    if relphi is None:
        relphis = rng.uniform(0.001, 0.999, size=num)
    else: 
        relphis = [relphi] * num
    
    return [phi_from_relphi(num_candidates, relphi=relphis[n]) for n in range(num)]


def generate_profiles2(num_candidates, num_voters, **kwargs): 

    if 'probmodel' in kwargs:
        probmodel = kwargs['probmodel']
    else: 
        probmodel = "impartial"

    if 'seed' in kwargs:
        seed = kwargs['seed']
    else: 
        seed = None
    
    if 'num_profiles' in kwargs:
        num_profiles = kwargs['num_profiles']
    else: 
        num_profiles = 1
    
    if 'anonymize' in kwargs: 
        anonymize = kwargs['anonymize']
    else:
        anonymize = False

    if probmodel == "IC" or probmodel == 'impartial': 
        
        profs = [Profile(impartial(num_voters, 
                                   num_candidates, 
                                   seed=seed))  
                                   for _ in range(num_profiles)]
    
    elif probmodel == "IAC" or probmodel == 'impartial_anonymous': 
        
        profs = [Profile(impartial_anonymous(num_voters, 
                                             num_candidates, 
                                             seed=seed)) 
                                             for _ in range(num_profiles)]

    elif probmodel == "MALLOWS" or probmodel == 'mallows':

        if 'phi' in kwargs: 
            phi = kwargs['phi']
        else:
            phi = 0.5
            
        if 'normalise_phi' in kwargs: 
            normalise_phi = kwargs['normalise_phi']
        else:
            normalise_phi = False

        if 'central_vote' in kwargs: 
            central_vote = kwargs['central_vote']
        else:
            central_vote = None

        profs = [Profile(mallows(num_voters,
                                 num_candidates, 
                                 phi,
                                 normalise_phi=normalise_phi,
                                 central_vote=central_vote,
                                 seed=seed)) 
                                 for _ in range(num_profiles)]

    elif probmodel == "MALLOWS-RELPHI":

        if 'relphi' in kwargs: 
            relphi = kwargs['relphi']
        else:
            relphi = None
            
        if 'normalise_phi' in kwargs: 
            normalise_phi = kwargs['normalise_phi']
        else:
            normalise_phi = False

        if 'central_vote' in kwargs: 
            central_vote = kwargs['central_vote']
        else:
            central_vote = None

        phis = phis_from_relphi(num_candidates, num_profiles, relphi, seed=seed)

        profs = [Profile(mallows(num_voters,
                                 num_candidates, 
                                 phis[n],
                                 normalise_phi=normalise_phi,
                                 central_vote=central_vote,
                                 seed=seed)) 
                                 for n in range(num_profiles)]


    elif probmodel == "URN" or probmodel == 'urn': 

        if 'alpha' in kwargs: 
            alpha = kwargs['alpha']
        else:
            alpha = 1.0
            
        profs = [Profile(urn(num_voters,
                             num_candidates, 
                             alpha,
                             seed=seed)) 
                             for _ in range(num_profiles)]


    elif probmodel == "URN-10":
        
        alpha = 10
        profs = [Profile(urn(num_voters,
                             num_candidates, 
                             alpha,
                             seed=seed)) 
                             for _ in range(num_profiles)]
    
    elif probmodel == "URN-0.3":
        
        alpha = round(math.factorial(num_candidates) * 0.3)
        profs = [Profile(urn(num_voters,
                             num_candidates, 
                             alpha,
                             seed=seed)) 
                             for _ in range(num_profiles)]
        
    elif probmodel == "URN-R":
        
        rng = np.random.default_rng(seed)
        alpha = round(math.factorial(num_candidates) * gamma.rvs(0.8, random_state=rng))
        profs = [Profile(urn(num_voters,
                             num_candidates, 
                             alpha,
                             seed=seed)) 
                             for _ in range(num_profiles)]

    elif probmodel == "plackett_luce":
        
        if 'alphas' not in kwargs:
            print("Error: alphas parameter missing.  A value must be specified for each candidate indicating their relative quality.")
            #RaiseValueError()
        else:
            alphas = kwargs['alphas']

        profs = [Profile(plackett_luce(num_voters,
                                       num_candidates, 
                                       alphas,
                                       seed=seed)) 
                                       for _ in range(num_profiles)]

    elif probmodel == "didi":
        
        if 'alphas' not in kwargs:
            print("Error: alphas parameter missing.  A value must be specified for each candidate indicating each candidate's quality.")
            #RaiseValueError()
        else:
            alphas = kwargs['alphas']

        profs = [Profile(didi(num_voters,
                              num_candidates, 
                              alphas,
                              seed=seed)) 
                              for _ in range(num_profiles)]

    elif probmodel == "stratification":
        
        if 'weight' not in kwargs:
            print("Error: weight parameter missing.  The weight parameter specifies the size of the upper class of candidates.")
            #RaiseValueError()
        else:
            weight = kwargs['weight']

        profs = [Profile(stratification(num_voters,
                                        num_candidates, 
                                        weight,
                                        seed=seed)) 
                                        for _ in range(num_profiles)]
    
    elif probmodel == "single_peaked_conitzer":
        
        profs = [Profile(single_peaked_conitzer(num_voters,
                                                num_candidates, 
                                                seed=seed)) 
                                                for _ in range(num_profiles)]
    
    elif probmodel == "SinglePeaked" or probmodel == "single_peaked_walsh":
        
        profs = [Profile(single_peaked_walsh(num_voters,
                                             num_candidates, 
                                             seed=seed)) 
                                             for _ in range(num_profiles)]

    elif probmodel == "single_peaked_circle":
        
        profs = [Profile(single_peaked_circle(num_voters,
                                              num_candidates, 
                                              seed=seed)) 
                                              for _ in range(num_profiles)]

    elif probmodel == "single_crossing":
        
        profs = [Profile(single_crossing(num_voters,
                                         num_candidates, 
                                         seed=seed)) 
                                         for _ in range(num_profiles)]
        
    elif probmodel == "euclidean":
        euclidean_models = {
            "uniform": EuclideanSpace.UNIFORM,
            "ball": EuclideanSpace.BALL,
            "gaussian": EuclideanSpace.GAUSSIAN,
            "sphere": EuclideanSpace.SPHERE,
        }

        if 'space' in kwargs:
            space = kwargs['space']
        else:
            space = "uniform"

        if 'dimension' in kwargs:
            dimension = kwargs['dimension']
        else:
            dimension = 2

        profs = [Profile(euclidean(num_voters,
                                   num_candidates, 
                                   space = euclidean_models[space],
                                   dimension = dimension,                                
                                   seed=seed)) 
                                   for _ in range(num_profiles)]

    if anonymize: 
        profs = [prof.anonymize() for prof in profs]
    return profs[0] if num_profiles == 1 else profs

def generate_profile_from_probability_models(
        num_candidates, 
        num_voters, 
        probmodels, 
        weights=None,
        seed=None, 
        num_profiles=1, 
        anonymize=False, 
        **kwargs):
    pass



In [ ]:
# Given the number m of candidates and a absolute number of expected swaps exp_abs, this function returns a value of phi such that in a vote sampled from Mallows model with this parameter the expected number of swaps is exp_abs
def phi_from_relphi(num_candidates, relphi=None, seed=None):

    rng = np.random.default_rng(seed)
    if relphi is None:
        relphi = rng.uniform(0.001, 0.999, )
    if relphi == 1:
        return 1
    exp_abs = relphi * (num_candidates * (num_candidates - 1)) / 4
    low = 0
    high = 1
    while low <= high:
        mid = (high + low) / 2
        cur = find_expected_number_of_swaps(num_candidates, mid)
        if abs(cur - exp_abs) < 1e-5:
            return mid
        # If x is greater, ignore left half
        if cur < exp_abs:
            low = mid

        # If x is smaller, ignore right half
        elif cur > exp_abs:
            high = mid

    # If we reach here, then the element was not present
    return -1



In [79]:
phis_from_relphi(4, 10, seed=10) for _ in range(10)



[0.9396591186523438,
 0.9396591186523438,
 0.9396591186523438,
 0.9396591186523438,
 0.9396591186523438,
 0.9396591186523438,
 0.9396591186523438,
 0.9396591186523438,
 0.9396591186523438,
 0.9396591186523438]

In [56]:
generate_profiles2(3, 4,  anonymize=True, central_vote=[2, 1, 0], probmodel="mallows", phi=1.0).display()



+---+---+---+
| 1 | 2 | 1 |
+---+---+---+
| 0 | 0 | 2 |
| 2 | 1 | 1 |
| 1 | 2 | 0 |
+---+---+---+


In [3]:
prob_models = {
    "IC": {
        "func": create_rankings_urn,
        "param": 0,
    },  # IC model is the Urn model with alpha=0
    "IAC": {"func": create_rankings_urn, "param": 1},  # IAC model is urn with alpha=1
    "MALLOWS-0.8": {"func": create_rankings_mallows, "param": 0.8},
    "MALLOWS-0.2": {"func": create_rankings_mallows, "param": 0.2},
    "MALLOWS-R": {
        "func": create_rankings_mallows,
        "param": lambda nc: random.uniform(0.001, 0.999),
    },
    "MALLOWS-RELPHI-0.4": {
        "func": create_rankings_mallows,
        "param": lambda nc: phi_from_relphi(nc, 0.4),
    },
    "MALLOWS-RELPHI-0.375": {
        "func": create_rankings_mallows,
        "param": lambda nc: phi_from_relphi(nc, 0.375),
    },
    "MALLOWS-RELPHI-0": {
        "func": create_rankings_mallows,
        "param": lambda nc: phi_from_relphi(nc, 0),
    },
    "MALLOWS-RELPHI-1": {
        "func": create_rankings_mallows,
        "param": lambda nc: phi_from_relphi(nc, 1),
    },
    "MALLOWS-RELPHI-R": {
        "func": create_rankings_mallows,
        "param": lambda nc: phi_from_relphi(nc),
    },
    "MALLOWS-RELPHI-R2": {
        "func": create_rankings_mallows,
        "param": lambda nc: phi_from_relphi(nc, random.uniform(0.001, 0.5)),
    },
    "MALLOWS_2REF-0.8": {"func": create_rankings_mallows_two_rankings, "param": 0.8},
    "MALLOWS_2REF-RELPHI-R": {
        "func": create_rankings_mallows_two_rankings,
        "param": lambda nc: phi_from_relphi(nc),
    },
    "MALLOWS_2REF-RELPHI-R2": {
        "func": create_rankings_mallows_two_rankings,
        "param": lambda nc: phi_from_relphi(nc, random.uniform(0.001, 0.5)),
    },
    "URN-10": {"func": create_rankings_urn, "param": 10},
    "URN-0.1": {
        "func": create_rankings_urn,
        "param": lambda nc: round(math.factorial(nc) * 0.1),
    },
    "URN-0.3": {
        "func": create_rankings_urn,
        "param": lambda nc: round(math.factorial(nc) * 0.3),
    },
    "URN-R": {
        "func": create_rankings_urn,
        "param": lambda nc: round(math.factorial(nc) * gamma.rvs(0.8)),
    },
    "SinglePeaked": {"func": create_rankings_single_peaked, "param": None},
}


def get_replacement(num_cands, param):
    return int(num_cands * param)


def generate_profile(num_cands, num_voters, probmod="IC", probmod_param=None):
    """Generate a :class:`Profile` with ``num_cands`` candidates and ``num_voters`` voters using the  probabilistic model ``probmod`` (with parameter ``probmod_param``).

    :param num_cands: the number of candidates in the profile
    :type num_cands: int
    :param num_voters: the number of voters in the profile
    :type num_voters: int
    :param probmod: the probability model used to generate the :class:`Profile`
    :type probmod: str, optional (default "IC")
    :param probmod_param: a parameter to the probability model
    :type probmod_param: number or function, optional
    :returns: A profile of strict linear orders
    :rtype: Profile


    :Example:

    .. exec_code::

        from pref_voting.generate_profiles import generate_profile
        prof = generate_profile(4, 10) # default is probmod is IC
        prof.display()
        prof = generate_profile(4, 10, probmod="IAC")
        prof.display()
        prof = generate_profile(4, 10, probmod="URN-0.3")
        prof.display()
        prof = generate_profile(4, 10, probmod="MALLOWS-R")
        prof.display()
        prof = generate_profile(4, 10, probmod="MALLOWS-RELPHI-0.375")
        prof.display()
        prof = generate_profile(4, 10, probmod="SinglePeaked")
        prof.display()

    :Possible Values of probmod:

    - "IC" (Impartial Culture);
    - "IAC" (Impartial Anonymous Culture);
    - "URN-10" (URN model with :math:`\\alpha=10`), "URN-0.1"  (URN model with :math:`\\alpha=0.1*num\_cands!`), "URN-0.3" (URN model with :math:`\\alpha=0.3*num\_cands!`), "URN-R" (URN model with randomly chosen :math:`\\alpha`);
    - "MALLOWS-0.8" (Mallows model with :math:`\\phi=0.8`), "MALLOWS-0.2" (Mallows model with :math:`\\phi=0.2`), "MALLOWS-R" (Mallows model with :math:`\\phi` randomly chosen between 0 and 1);
    - "MALLOWS-RELPHI-0.4" (Mallows model with :math:`\\phi` defined from ``num_cands`` and the relphi value of 0.4), "MALLOWS-RELPHI-0.375" (Mallows model with :math:`\\phi` defined from ``num_cands`` and the relphi value of 0.375), "MALLOWS-RELPHI-0" (Mallows model with :math:`\\phi` defined from ``num_cands`` and the relphi value of 0),  "MALLOWS-RELPHI-1" (Mallows model with :math:`\\phi` defined from ``num_cands`` and the relphi value of 1), (Mallows model with :math:`\\phi` defined from ``num_cands`` and the relphi value randomly chosen based on the number of candidates), "MALLOWS-RELPHI-R2" (Mallows model with :math:`\\phi` defined from ``num_cands`` and the relphi value randomly chosen), "MALLOWS_2REF-0.8" (Mallows model with 2 reference rankings and :math:`\\phi = 0.8`),
    - "MALLOWS_2REF-RELPHI-R": (Mallows model with 2 reference rankings and :math:`\\phi` defined from ``num_cands`` and a randomly chosen relphi value based on the number of candidates), "MALLOWS_2REF-RELPHI-R2"(Mallows model with 2 reference rankings and :math:`\\phi` defined from ``num_cands`` and a randomly chosen relphi value); and
    - "SinglePeaked" (Single Peaked)

    In addition, you can customize the probability model used to generate a profile as follows:

    - ``probmod`` is "URN" and ``probmod_param`` is either a number or a function :math:`f` and the parameter is defined by applying :math:`f` to the number of candidates.

    - ``probmod`` is "MALLOWS" and ``probmod_param`` is either a number or a function :math:`f` and the parameter is defined by applying :math:`f` to the number of candidates.

    - ``probmod`` is "MALLOWS_2REF" and ``probmod_param`` is either a number or a function :math:`f` and the parameter is defined by applying :math:`f` to the number of candidates.

    :Example:

    .. exec_code::

        import math
        from pref_voting.generate_profiles import generate_profile
        prof = generate_profile(4, 10, probmod="URN", probmod_param=5)
        prof.display()
        prof = generate_profile(4, 10, probmod="MALLOWS", probmod_param=0.5)
        prof.display()
        prof = generate_profile(4, 10, probmod="MALLOWS_2REF", probmod_param=0.5)
        prof.display()
        prof = generate_profile(4, 10, probmod="URN", probmod_param=lambda nc: math.factorial(nc) * 0.5)
    """

    if probmod in prob_models.keys():

        create_rankings = prob_models[probmod]["func"]
        _probmod_param = prob_models[probmod]["param"]

    elif probmod == "Spatial":

        num_dims = probmod_param[0] if probmod_param is not None else 2
        voter_utility = probmod_param[1] if probmod_param is not None else linear_utility

        sprof = generate_spatial_profile(num_cands, num_voters, num_dims=num_dims)

        return sprof.to_utility_profile(utility_function=voter_utility).to_ranking_profile()
    elif probmod == "URN":

        create_rankings = create_rankings_urn
        _probmod_param = probmod_param if probmod_param is not None else 0

    elif probmod == "MALLOWS":

        create_rankings = create_rankings_mallows
        _probmod_param = probmod_param if probmod_param is not None else 1

    elif probmod == "MALLOWS_2REF":

        create_rankings = create_rankings_mallows_two_rankings
        _probmod_param = probmod_param if probmod_param is not None else 1

    else:
        print(f"{probmod}: Probability model not implemented, no profile generated.")
        return None

    probmod_param = (
        _probmod_param(num_cands) if callable(_probmod_param) else _probmod_param
    )

    rankings, rcounts = create_rankings(num_cands, num_voters, probmod_param)

    return Profile(rankings, rcounts=rcounts)



In [88]:
from prefsampling.ordinal.impartial import impartial

rng=np.random.default_rng(10012)
print(rng.uniform(0.001, 0.999, size=10))

[0.09244842 0.20017731 0.85072899 0.10402968 0.23226742 0.61256133
 0.31242714 0.78777006 0.75112153 0.03093063]


In [97]:
phis_from_relphi(4, 10, seed=10)

[0.9396591186523438,
 0.1966094970703125,
 0.7854080200195312,
 0.14348602294921875,
 0.46932220458984375,
 0.13117218017578125,
 0.6374435424804688,
 0.8005218505859375,
 0.39048004150390625,
 0.9408645629882812]

In [17]:
for i in range(10): 
    print(impartial(3, 5), "\n")

[[3 0 2 1 4]
 [1 3 2 4 0]
 [2 1 4 3 0]] 

[[3 0 2 4 1]
 [2 4 1 0 3]
 [4 0 3 2 1]] 

[[1 3 2 4 0]
 [1 3 2 4 0]
 [2 3 1 0 4]] 

[[0 3 2 4 1]
 [4 0 1 2 3]
 [3 0 4 2 1]] 

[[0 3 2 1 4]
 [3 4 1 2 0]
 [3 1 4 2 0]] 

[[2 0 1 4 3]
 [1 2 3 4 0]
 [0 4 3 2 1]] 

[[3 2 1 0 4]
 [4 2 0 3 1]
 [0 3 2 4 1]] 

[[4 0 3 2 1]
 [3 1 4 0 2]
 [2 3 1 0 4]] 

[[1 3 4 0 2]
 [0 4 3 2 1]
 [2 1 4 3 0]] 

[[4 1 2 3 0]
 [0 4 2 3 1]
 [3 0 1 4 2]] 

